# Cupid Email Sender v1
### by Walter Shen (Imperial College London 3rd year maths undergraduate)
### email shen.walter@yahoo.com if bugs

last edited 14/02/2022

Description: take in the excel file outputted from survey results (ex. from here https://forms.office.com/r/uFxjriiY3p) and send out individualized Valentine's Day emails with user submitted information.

But code may not be perfect, things may... break up (get it? terrible Valentine's Day puns)

Originally coded for RCSU Valentine's Day 2022 anonymous message service.

Setup:
- change sender_email and smtp_server/port as needed
- make sure reading in correct excel file
- ensure correct column names are called (this will vary according to your survey)
- check password file is correct (aka password stored in plaintext .txt file, which I am aware is very insecure)
- consider changing the html file, to make it prettier (my HTML/CSS skills are limited)

Other:
- on form, ensure recipeint email and message boxes are set as REQUIRED to save time
- will have to manually prune any weirdly formatted emails + sketchy messages
- manually fix any unsendable emails (ie. incorrect address)
- consider doing it in batches to troubleshoot anything that goes wrong in smaller segments


In [ ]:
sender_email = "EMAIL@ic.ac.uk" # email to send from here

In [ ]:
import pandas as pd

# read in excel file output from Microsoft Forms
data= pd.read_excel("RCSU_batch_1.xlsx")

# show first two entries
data[0:2]

In [ ]:
# remove the spaces in the column names
data.columns = data.columns.str.replace(' ', '')
data.columns

# in this instance we will be using ID, RecipientName, RecipientEmail, and Message

In [ ]:
# email setup
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

# server details
smtp_server = "smtp.office365.com"#"smtp.gmail.com" # smtp.office365.com
# note: for google, had to change security permissions https://www.google.com/settings/security/lesssecureapps
port = 587 

In [ ]:
# make sure this cell has no output!
f = open("cupid_key2.txt", "r") # cupid_key.txt has the password stored in plaintext...
password=f.readline()
f.close()

# create a secure SSL context
context = ssl.create_default_context()

In [ ]:
# test the login
sending = smtplib.SMTP(smtp_server,port)
sending.ehlo()
sending.starttls()
sending.login(sender_email,password)
print("logged in!")

In [ ]:
# startup the email sending
sending = smtplib.SMTP(smtp_server,port)
sending.ehlo()
sending.starttls()
sending.login(sender_email,password)
print("logged in!")

n = len(data.ID) # number of emails that have to be sent
for i in range(n):
    # Create message container
    msg = MIMEMultipart('alternative')
    
    # Fetch the message body and format the line breaks
    message = data.Message[i].replace("\n","<br>")
    
    # recipient info (almost certain i spelled recipient wrong...)
    to_email = data.RecipientEmail[i]
    recpientname= data.RecipientName[i]
    
    # subject line
    subject = f"{recpientname}, You have a Valentine's day message ❤️"
    
    # html format the body + other text, make it look pretty
    # preview what message looks like here: https://letterrcsu.w3spaces.com/saved-from-Tryit-2022-02-12.html?bypass-cache=28471499
    html =f"""
<html>
  <head>
    <link href="https://fonts.googleapis.com/css2?family=Caveat&family=Dancing+Script&display=swap" rel="stylesheet">
  </head>
<body>
<div style= 'max-width: 500px;margin: auto; border: 4px dotted #f8b8c0; background-color: #ffcccb; padding: 0px 25px 25px 25px;'>
  <p style='color: #a838c9; text-align: center; font-size: 24px; font-family: "Dancing Script", cursive;'><strong>A Valentine's message for you</strong></p>
  <p style='color: #4c1a5c; font-size: 18px; font-family: "Caveat", cursive;'><strong>TO: {recpientname}<br>FROM: Secret Valentine</strong></p>
  <p style='color: black; font-size: 16px; font-family: "Caveat", cursive;'>{message}</p>
</div>

<p style='color: red; text-align: center; font: 14px "Helvetica Neue"'><strong>Happy Valentine's Day!<br>From Your RCSU Cupids xxx</strong></p>
<p style='color: #ee2a7b; text-align: center; font: 10px "Helvetica Neue"'><strong>14.02.2022</strong></p>
<p style='font: 12px "Helvetica Neue"; text-align: center;'><em>DISCLAIMER: We (the Royal College of Science Union, RCSU) are not responsible for the content of the message; we are only passing on the message from an automated and fully anonymized system.</em></p> 
</body>
</html>
"""
    
    # record MIME html
    message_html = MIMEText(html, 'html')

    # attach to message container.
    msg.attach(message_html)
    
    # add email details to message container
    msg['Subject'] = subject
    reciepients = [to_email, sender_email]
    msg['To'] = ', '.join(reciepients)
    msg['From'] = sender_email
    
    # sent out the email (also send copy to self)
    sending.sendmail(sender_email, to_email, msg.as_string())
    print(f"sent mail number {i+1} of {n} to {to_email}")
    
print("finished sending emails for batch")

In [1]:
# clears password for security
password = 'xx'